In [ ]:
config_file = 'delineation/configs/dislocation_matching_disp_and_warp_and var_joint_home.yml'
aug_config_file = 'delineation/configs/aug.yml'
import torch
from delineation.configs.defaults_segmentation import _C as cfg
from delineation.datasets import make_data_loader
from delineation.models import build_model_list
from delineation.utils import settings, cost_volume_helpers
from delineation.utils.settings import evaluate_results

from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

import yaml

cfg.merge_from_file(config_file)
with open(aug_config_file, 'r') as ymlfile:
    cfg_aug = yaml.load(aug_config_file)

_device = settings.initialize_cuda_and_logging(cfg)  # '_device' is GLOBAL VAR

train_loader, val_loader = make_data_loader(cfg, cfg_aug)
seg_model, model = build_model_list(cfg, True)

In [ ]:
seg_model.eval()
model.eval()
total_test_loss = 0
pix1_err_m, pix3_err_m, pix5_err_m, epe_m, count = 0, 0, 0, 0, 0
from delineation.utils import settings, cost_volume_helpers

for batch_idx, (l_aug, r_aug, l_gt_aug, r_gt_aug, l, r, lgt, rgt, dlgt, l_name) in (enumerate(val_loader)):
        indices = cost_volume_helpers.volume_indices(2 * cfg.TRAINING.MAXDISP, len(l),
                                                     cfg.TRAINING.HEIGHT, cfg.TRAINING.WIDTH, _device)

        with torch.no_grad():
            
            l, r, lgt, rgt, dlgt = l.to(_device), r.to(_device), lgt.to(_device), rgt.to(_device), dlgt.to(_device)
            with torch.no_grad():

                l_segmap, l_seg = seg_model(l)
                r_segmap, r_seg = seg_model(r)

            if abs(dlgt.max()>32) or abs(dlgt.min()>32):
                continue
                
            print(l_name)
            dl_scores = model(l_segmap, r_segmap)
            
            dl_ = F.softmax(-dl_scores, 2)
            dl = torch.sum(dl_.mul(indices), 2) - cfg.TRAINING.MAXDISP
            for i in range(0, len(dlgt)):
                pix1_err, pix3_err, pix5_err, epe = evaluate_results(dlgt[i], dl[i], lgt[i])
                print(pix1_err, pix3_err, pix5_err, epe, dlgt.min(), dlgt.max())
                pix1_err_m += pix1_err
                pix3_err_m += pix3_err
                pix5_err_m += pix5_err
                epe_m += epe
                count += 1
            torch.cuda.empty_cache()
            
            l_seg = l_seg.cpu().numpy()>0.3

            lgt = lgt.cpu().numpy()
            dl = dl.detach().permute(0, 2, 3, 1).cpu().numpy()
            dlgt = dlgt.unsqueeze(1).detach().permute(0, 2, 3, 1).cpu().numpy()

            for i in tqdm(range(len(dl))):
                vol = cost_volume_helpers.back_project_numpy(l_seg[i,0,:,:], dl[i,:,:,:], cfg.TRAINING.MAXDISP, mode='two-sided')
                vol_gt = cost_volume_helpers.back_project_numpy(lgt[i,0,:,:], dlgt[i,:,:,:], cfg.TRAINING.MAXDISP, mode='two-sided')

                for idx, rotation_angle in enumerate(range(-20, 20, 1)):
                    cost_volume_helpers.visualize_volume(l_name[i], vol[0, :, :, :],
                                                         rotation_angle,
                                                         cfg.LOGGING.LOG_DIR,
                                                         mode='scatter',
                                                         save_ext=str(idx)+'.png',
                                                         plot=False)

                    cost_volume_helpers.visualize_volume(l_name[i], vol_gt[0, :, :, :],
                                                         rotation_angle,
                                                         cfg.LOGGING.LOG_DIR+'_gt',
                                                         mode='scatter',
                                                         save_ext=str(idx)+'.png',
                                                         plot=False)

                torch.cuda.empty_cache()

            
print('here')
values = pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count
print('Mean per dataset: {}, {}, {}, {}'.format(pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count))

('1116_1116 DPC DF4 71.0 kx DF4-B_LEFT.png',)


  0%|          | 0/1 [00:00<?, ?it/s]

0.22878228782287824 0.0011741026501174102 0.0 0.6691043 tensor(-4., device='cuda:0') tensor(2., device='cuda:0')
[[-3.1033134  -3.6813984  -3.6588173  ... -0.34099197 -1.2749367
  -1.9751167 ]
 [-3.3026924  -4.510624   -3.1874714  ...  0.6189232  -0.61956024
  -1.2960854 ]
 [-2.946085   -2.7505074  -2.2107105  ...  0.3240509   0.1601944
  -1.7193756 ]
 ...
 [-0.712759   -1.6679649  -2.2069607  ...  0.6357155   0.84165955
   1.841919  ]
 [ 0.43390656 -1.2754993  -1.5663204  ...  1.2310333  -0.11603928
  -0.03928375]
 [-0.10754013 -1.1020641  -2.0487175  ...  0.83685684 -0.04378128
   0.5284729 ]]
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
[[-3.1033134  -3.6813984  -3.6588173  ... -0.34099197 -1.2749367
  -1.9751167 ]
 [-3.3026924  -4.510624   -3.1874714  ...  0.6189232  -0.61956024
  -1.2960854 ]
 [-2.946085   -2.7505074  -2.2107105  ...  0.3240509   0.1601944
  -1.7193756 ]
 ...
 [-0.712759   -1.6679649  -2.20